In [1]:
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

In [2]:
def getLogID(log, sn, fault_time):
    day2sec = 24*60*60
    tmp = log[(log['sn']==sn)].copy(deep=True)
    tmp['delta'] = tmp['timestamp'].apply(lambda x: abs((fault_time-x).total_seconds()))
    tmp = tmp[tmp['delta'] <= tmp['delta'].min() + day2sec]
    
    tmp = tmp.sort_values('timestamp')
    
    LogID = tmp.index
    LogID = ' '.join(str(i) for i in LogID)
    return LogID

In [3]:
train = pd.read_csv('./dataset/v1/train.csv')
train['timestamp'] = pd.to_datetime(train['timestamp'])

label = pd.concat([pd.read_csv('./dataset/src/preliminary_train_label_dataset.csv'), 
                  pd.read_csv('./dataset/src/preliminary_train_label_dataset_s.csv')])
label['fault_time'] = pd.to_datetime(label['fault_time'])

label['LogID'] = label.progress_apply(lambda x: getLogID(train, x['sn'], x['fault_time']), axis=1)
label.to_csv('./dataset/v1/label.csv', index=False)

100%|█████████████████████████████████████| 16669/16669 [04:57<00:00, 56.00it/s]


In [4]:
test = pd.read_csv('./dataset/v1/test.csv')
test['timestamp'] = pd.to_datetime(test['timestamp'])

submit = pd.read_csv('./dataset/src/preliminary_submit_dataset_b.csv')
submit['fault_time'] = pd.to_datetime(submit['fault_time'])

submit['LogID'] = submit.progress_apply(lambda x: getLogID(test, x['sn'], x['fault_time']), axis=1)
submit.to_csv('./dataset/v1/submit.csv', index=False)

100%|██████████████████████████████████████| 3030/3030 [00:03<00:00, 983.97it/s]
